In [152]:
include("../Algebra/Gradients.jl")
include("../Algebra/Hamiltonians.jl")
include("../Algebra/Matrices.jl")
include("../Algebra/Propagators.jl")
include("../Amplitudes/Chebyshev.jl")
include("../Costs/Costs.jl")
include("../Gates/Xgate.jl")

U2 (generic function with 1 method)

In [153]:
using LinearAlgebra, Optim, Plots
using BenchmarkTools, Distributions

In [154]:
ω_c = 5
ω_q = 2
χ = 3e-3
N = 6
amp_c = 1
amp_q = 1

1

In [155]:
I_qubit = Matrix(I,2,2)
I_cavity = Matrix(I,N,N)
X_gate = kron(x_gate(N, Array[[2,5]]), I_qubit)

#X_gate = kron(x_gate(N, Array[[2,3], [4,5]]), I_qubit)

θ = π/2.5
RZ = kron(Rz(N, Array[[3,4]], θ/2), I_qubit)
RX = kron(Rx(N, Array[[3,4]], θ/4), I_qubit)
Udue = kron(U2(N, Array[[3,4]], [θ/2 θ/3 θ/4] ), I_qubit)

sx = kron(I_cavity, [0 1; 1 0])
sy = kron(I_cavity, [0 -1im; 1im 0])

ψ_initial = rand(N)
ψ_initial = kron(ψ_initial/norm(ψ_initial), [0,1])

12-element Vector{Float64}:
 0.0
 0.5817205390250986
 0.0
 0.6236041889846737
 0.0
 0.36617504412919033
 0.0
 0.3409463871340398
 0.0
 0.11335099509846902
 0.0
 0.09768305904574184

In [156]:
ψ_final_u = Udue * ψ_initial
ψ_final_x = RX * ψ_initial
ψ_final_x_gate = X_gate * ψ_initial

12-element Vector{Float64}:
 0.0
 0.5817205390250986
 0.0
 0.11335099509846902
 0.0
 0.36617504412919033
 0.0
 0.3409463871340398
 0.0
 0.6236041889846737
 0.0
 0.09768305904574184

In [157]:
# generating matrices
a,adag,sp,sm,sz = generate_matrices(N)

([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0 1 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 1; 0 0 … 0 0], [0 0 … 0 0; 1 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 1 0], [1 0 … 0 0; 0 -1 … 0 0; … ; 0 0 … 1 0; 0 0 … 0 -1])

In [158]:
disp(α) = exp((α * adag - α' * a))
ECD(β) = disp(β / 2) * sp + disp(- β / 2) * sm
ROT(ϕ, θ) = exp(-1im * θ / 2 * ( sx * cos(ϕ) + sy * sin(ϕ) )) 
snap(x) = kron(Diagonal(exp.(-1im.*x)),I_qubit)

snap (generic function with 1 method)

In [159]:
function cost(coefficients, unitary, N)

    L = length(coefficients)
    β_i = coefficients[1: Int(L/4)]
    β_r = coefficients[Int(L/4) + 1 : Int(L/2)]

    θs = coefficients[Int(L/2) + 1 : Int(3 * L/4)]
    ϕs = coefficients[Int(3 * L/4) + 1 : end]

    propagator = Matrix{ComplexF64}(I,N,N)

    for i in 1:length(β_i)
        propagator = propagator * ROT(ϕs[i], θs[i]) * ECD(β_i[i] + β_r[i] * 1im)
    end

    return 1 - norm(tr(propagator*unitary') / N)^2
    
end

function cost_ecd_rot_ecd(coefficients, unitary, N)

    L = length(coefficients)
    β_i = coefficients[1: Int(L/4)]
    β_r = coefficients[Int(L/4) + 1 : Int(L/2)]

    θs = coefficients[Int(L/2) + 1 : Int(3 * L/4)]
    ϕs = coefficients[Int(3 * L/4) + 1 : end]

    propagator = Matrix{ComplexF64}(I,N,N)

    for i in 1:length(β_i)
        propagator = propagator * ECD(β_i[i] + β_r[i] * 1im) * ROT(ϕs[i], θs[i]) * ECD(β_i[i] + β_r[i] * 1im)'
    end

    return 1 - norm(tr(propagator*unitary') / N)^2
    
end

function gradient(coefficients, unitary, N, h, cost_before_increment)

    L = length(coefficients)
    gradient = zeros(L)

    for i in 1:L
        coeffs_copy = copy(coefficients)
        coeffs_copy[i] = coefficients[i] + h
        gradient[i] = (cost(coeffs_copy, unitary, N) - cost_before_increment) / h
    end

    return gradient

end

function gate_composition(coefficients, N, l_coeff)

    L = length(coefficients)
    β_i = coefficients[1: Int(L/4)]
    β_r = coefficients[Int(L/4) + 1 : Int(L/2)]

    θs = coefficients[Int(L/2) + 1 : Int(3 * L/4)]
    ϕs = coefficients[Int(3 * L/4) + 1 : end]

    propagator = Matrix{ComplexF64}(I,N,N)

    for i in 1:length(β_i)
        propagator = propagator * ROT(ϕs[i], θs[i]) * ECD(β_i[i] + β_r[i] * 1im)
    end

    return propagator
    
end

function cost_state_prep(coefficients, N, ψ_n, ψ_m)

    L = length(coefficients)
    β_i = coefficients[1: Int(L/4)]
    β_r = coefficients[Int(L/4) + 1 : Int(L/2)]

    θs = coefficients[Int(L/2) + 1 : Int(3 * L/4)]
    ϕs = coefficients[Int(3 * L/4) + 1 : end]

    propagator = Matrix{ComplexF64}(I,N,N)

    for i in 1:length(β_i)
        propagator = propagator * ROT(ϕs[i], θs[i]) * ECD(β_i[i] + β_r[i] * 1im)
    end

    return 1 - norm(ψ_m' * propagator * ψ_n)^2
    
end


function gradient_state_prep(coefficients, N, h, cost_before_increment, ψ_n, ψ_m)

    L = length(coefficients)
    gradient = zeros(L)

    for i in 1:L
        coeffs_copy = copy(coefficients)
        coeffs_copy[i] = coefficients[i] + h
        gradient[i] = (cost_state_prep(coeffs_copy, N, ψ_n, ψ_m) - cost_before_increment) / h
    end

    return gradient

end

gradient_state_prep (generic function with 1 method)

In [160]:
snapgate = snap(rand(N))

12×12 Matrix{ComplexF64}:
 0.925156-0.379587im       0.0-0.0im       …       0.0+0.0im
      0.0-0.0im       0.925156-0.379587im          0.0+0.0im
      0.0+0.0im            0.0+0.0im               0.0+0.0im
      0.0+0.0im            0.0+0.0im               0.0+0.0im
      0.0+0.0im            0.0+0.0im               0.0+0.0im
      0.0+0.0im            0.0+0.0im       …       0.0+0.0im
      0.0+0.0im            0.0+0.0im               0.0+0.0im
      0.0+0.0im            0.0+0.0im               0.0+0.0im
      0.0+0.0im            0.0+0.0im               0.0+0.0im
      0.0+0.0im            0.0+0.0im               0.0+0.0im
      0.0+0.0im            0.0+0.0im       …       0.0-0.0im
      0.0+0.0im            0.0+0.0im          0.986348-0.164675im

In [161]:
f(x) = cost(x, RX, N * 2)
g(x) = cost_state_prep(x, N * 2, ψ_initial, ψ_final_x_gate)

g (generic function with 1 method)

In [162]:
costs = []
for depth in 18:6:36
    it_costs = []
    for iteration in 1:4
        println("blocks: ", depth," iterations :", iteration)
        l_coeff = depth * 4
        coefficients = zeros(l_coeff)
        # coefficients[1:Int(l_coeff / 2)] = [opt_coefficients[1:Int(length(opt_coefficients)/2)]..., (rand(Int(l_coeff / 2 - length(opt_coefficients)/2))*2 .-1)...]
        # coefficients[Int(l_coeff / 2) + 1:end] = [opt_coefficients[Int(length(opt_coefficients)/2)+1:end]..., zeros(Int(l_coeff / 2 - length(opt_coefficients)/2))...]
        coefficients[1:Int(l_coeff / 2)] = rand(Int(l_coeff / 2))*2 .-1
        coefficients[Int(l_coeff / 2) + 1:end] = rand(Int(l_coeff / 2)) * 2 * pi .- pi
        res = Optim.optimize(f, coefficients , LBFGS(), Optim.Options(show_trace=true, show_every=100, iterations = 1000))
        push!(it_costs,Optim.minimum(res))
        println(Optim.minimum(res))
        println(Optim.minimizer(res))
    end
    println(minimum(it_costs))
    push!(costs,minimum(it_costs))
end

blocks: 18 iterations :1
Iter     Function value   Gradient norm 


     0     9.908703e-01     2.902903e-02
 * time: 0.00010395050048828125


   100     1.526057e-01     1.466237e-02
 * time: 40.79516005516052


   200     8.955287e-02     3.861355e-03
 * time: 85.4905149936676


   300     6.052494e-02     1.257083e-02
 * time: 130.27053499221802


...

0.0026459547066395306
[0.31002431252104024, 0.3649750075054139, 0.7293983070758345, -0.1416365625739027, -1.293677104661913, -0.4200972290442632, -0.8916597377217215, -0.037186689093815686, 0.8924852956439758, -0.3963879945042835, -0.23982267248747247, 0.04701683375621748, 0.24255969241679323, 0.12761017318346143, -0.07980986244323177, 0.1964296905886447, 1.4610422539305075, 0.3724499118787606, -0.3477577072135677, -1.3155663928829593, -0.10324490268187583, -0.39061673649828144, 0.38277550483467154, 0.05051592775353075, -1.5430656142797379, -1.613398497700611, 0.9026463130593959, -0.1644279459155559, -0.953906395517016, -0.3462354450239098, -0.6265215010895047, -0.03593155760156625, 1.0142960600258677, 1.3466038217026466, 1.5262503484425696, -0.16717571770117548, -1.2366866873498505, -0.6297854091079967, 0.2642812179272193, 0.18596952553022408, 0.7004554634057037, -0.5312953630890275, 0.5323555317531139, -0.6435019350925779, -0.11128382235661485, -0.2866229406706452, -0.640057859902969

Iter     Function value   Gradient norm 
     0     9.983415e-01     1.030788e-02
 * time: 0.00014400482177734375


   100     1.549235e-02     1.221563e-02
 * time: 123.59063720703125


   200     6.716701e-03     5.587543e-03
 * time: 255.8545880317688


InterruptException: InterruptException:

In [163]:
RX = kron(Rx(N, Array[[3,4]], θ*2.5), I_qubit)

12×12 Matrix{ComplexF64}:
 1.0+0.0im  0.0+0.0im  0.0+0.0im  …  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  1.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  1.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im  …  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     1.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im  …  0.0+0.0im  1.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im     0.0+0.0im  0.0+0.0im  1.0+0.0im

In [164]:
f(x) = cost(x, RX, N * 2)
costs = []
for depth in 18:6:36
    it_costs = []
    for iteration in 1:4
        println("blocks: ", depth," iterations :", iteration)
        l_coeff = depth * 4
        coefficients = zeros(l_coeff)
        # coefficients[1:Int(l_coeff / 2)] = [opt_coefficients[1:Int(length(opt_coefficients)/2)]..., (rand(Int(l_coeff / 2 - length(opt_coefficients)/2))*2 .-1)...]
        # coefficients[Int(l_coeff / 2) + 1:end] = [opt_coefficients[Int(length(opt_coefficients)/2)+1:end]..., zeros(Int(l_coeff / 2 - length(opt_coefficients)/2))...]
        coefficients[1:Int(l_coeff / 2)] = rand(Int(l_coeff / 2))*2 .-1
        coefficients[Int(l_coeff / 2) + 1:end] = rand(Int(l_coeff / 2)) * 2 * pi .- pi
        res = Optim.optimize(f, coefficients , LBFGS(), Optim.Options(show_trace=true, show_every=100, iterations = 1000))
        push!(it_costs,Optim.minimum(res))
        println(Optim.minimum(res))
        println(Optim.minimizer(res))
    end
    println(minimum(it_costs))
    push!(costs,minimum(it_costs))
end

blocks: 18 iterations :1
Iter     Function value   Gradient norm 


     0     9.887353e-01     3.780432e-02
 * time: 0.0001361370086669922


InterruptException: InterruptException:

In [165]:
println("X 2,5")

f(x) = cost(x, X_gate, N * 2)
costs = []
for depth in 18:6:36
    it_costs = []
    for iteration in 1:5
        println("blocks: ", depth," iterations :", iteration)
        l_coeff = depth * 4
        coefficients = zeros(l_coeff)
        # coefficients[1:Int(l_coeff / 2)] = [opt_coefficients[1:Int(length(opt_coefficients)/2)]..., (rand(Int(l_coeff / 2 - length(opt_coefficients)/2))*2 .-1)...]
        # coefficients[Int(l_coeff / 2) + 1:end] = [opt_coefficients[Int(length(opt_coefficients)/2)+1:end]..., zeros(Int(l_coeff / 2 - length(opt_coefficients)/2))...]
        coefficients[1:Int(l_coeff / 2)] = rand(Int(l_coeff / 2))*2 .-1
        coefficients[Int(l_coeff / 2) + 1:end] = rand(Int(l_coeff / 2)) * 2 * pi .- pi
        res = Optim.optimize(f, coefficients , LBFGS(), Optim.Options(show_trace=true, show_every=100, iterations = 1000))
        push!(it_costs,Optim.minimum(res))
        println(Optim.minimum(res))
        println(Optim.minimizer(res))
    end
    println(minimum(it_costs))
    push!(costs,minimum(it_costs))
end

X 2,5
blocks: 18 iterations :1
Iter     Function value   Gradient norm 


     0     9.979078e-01     6.615834e-03
 * time: 0.00011801719665527344


InterruptException: InterruptException:

In [147]:
X_gate = kron(x_gate(N, Array[[2,5]]), I_qubit)

12×12 Matrix{Float64}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0

# stoc gradient descent

In [149]:
costs_final = []
for depth in 64:8:64
    it_costs = []
    for iteration in 1:3
        l_coeff = depth * 4
        coefficients = zeros(l_coeff)
        coefficients[1:Int(l_coeff / 2)] = rand(Int(l_coeff / 2))*2 .-1
        coefficients[Int(l_coeff / 2) + 1:end] = rand(Int(l_coeff / 2)) * 2 * pi .- pi
        coefficients = [0.8690408632149678, -0.7236416401971805, -0.0908719850588177, 0.8258628776740167, 0.9056445769487957, -0.8220939210081029, 0.29601861853616424, 0.7813552229187792, 0.9837594907380149, -0.10686709645389225, -0.6656407250459772, 0.992278727387623, -0.4893461915705191, -1.093993962903741, 0.4845407107960842, -0.33857113975897773, -0.438797994554033, -0.5977365084149797, 0.20827466739249778, -0.8959109454265681, 0.5383716212207733, 1.3219169845803183, 0.8578064725920909, -0.7256810259845291, 0.37256259960242327, -1.2114316020995235, -0.282902962907945, -0.26590734759323503, 0.5548879566071089, 0.6803304120122661, -1.0549066677248888, 0.4013045273205048, 0.039560949181603355, 1.0529793828938236, -0.044290684794351255, -0.21790564460826214, 0.7641852601374161, 0.9310403877890154, -0.24519813978115026, 0.6528372052655407, -1.2521949229101696, -0.21682088705693697, 0.10911919627652335, -0.31451889968452584, 0.07428438971871276, 0.8168118972903352, -0.4306959382326172, -1.31201487771103, -0.5551956081923786, -0.24938592347753555, 0.7261072879386111, 0.3449918944429117, 1.0218022420859423, 0.9586632779953215, 0.9830015979456759, 1.1800344827793208, -0.6015729201693757, 0.703237921035128, -0.8659071933834304, 0.7405471254766851, 0.5502125342538493, -0.1279261558044507, 0.5436405284887846, 0.7561715320031175, -1.8294742583884145, 0.8274849373492085, -0.5023928927013781, -0.2106775708083146, -0.25623844263590434, -0.13700323024471564, -0.5427032401830472, 0.8481733035397572, 2.857123893083541, 0.6722772567617452, 3.3393904892971857, -2.7921664352181588, -1.4927954172559996, 0.3500341652427946, 2.178592973947075, 2.0048971146113073, -1.3350642362244973, 0.9711032282505817, 2.3816703973089663, -2.6138104747765247, 2.5134841942159674, -1.9047337097507626, 2.2260526473073146, -1.8796539188004344, 1.6517968966982153, 1.4620145034237324, 1.8366376463164558, -1.3614199467978896, -1.3067881525922014, -0.8271088962045482, -1.3523594955345797, -0.8216607978872826, -1.015386302411769, -1.4951025575413324, 2.199130944918348, -2.098541424724332, 1.8106459325712736, -2.7528680557326783, 1.434522172097638, 0.23235089810469334, -1.3427653914596476, -2.1865004319117882, -4.499311913607152, 3.5233665940145906, 0.11147763894762594, 1.7253123134995119, 0.3069539608771961, 0.7456729443236856, 0.9675539799184902, -2.0807423878078692, -0.21274877285924562, -2.4625967782327733, -0.8958201449217154, -0.16506248177386307, -0.7781344488152088, 0.5657089824636788, 0.49103015856219095, -1.893089322888448, 1.8586226474868552, 1.3367905090651682, -0.845322993521433, -2.414547922619251, -2.0653040809018646, 2.13700608386267, 0.6560460356595541, 1.2398223292845443, 2.962599533837159, 1.1325322497723143, 3.463842619105753, -3.938053691752728, -0.9569023982656585, 1.605238908260515, 0.6974820952130029, 2.5406564651431, 2.285077612710168, 0.027961409188739638, 1.0736422395589698, 2.4870006619787572, 0.692378916991639, -2.718696870022777]
        beta1 = 0.8
        beta2 = 0.999
        alpha = 0.01
        m = zeros(length(coefficients))
        v = zeros(length(coefficients))
        coeffs_stored = []
        costs = []
        cost_prev_it = cost(coefficients, RX, N*2)  #cost_state_prep(coefficients, N * 2, ψ_initial, ψ_final_x_gate) #

        for i in 1:1000
            grad = gradient(coefficients, RX, N * 2, 1e-8, cost_prev_it) #gradient_state_prep(coefficients, N*2, 1e-8, cost_prev_it, ψ_initial, ψ_final_x_gate) #
            for n in 1:length(coefficients)
                m[n] = beta1 * m[n] + ( 1 - beta1 ) * grad[n]
                v[n] = beta2 * v[n] + ( 1 - beta2 ) * grad[n]^2
                m_norm = m[n] / ( 1 - beta1^i )
                v_norm = v[n] / ( 1 - beta2^i )
                coefficients[n] = coefficients[n] - alpha * m_norm / (sqrt(v_norm) + 1e-8)
            end
            cost_prev_it = cost(coefficients, RX, N*2)# cost_state_prep(coefficients, N * 2, ψ_initial, ψ_final_x_gate) #
            push!(costs, cost_prev_it)

            if i % 50 == 0
                if norm(grad) < 1e-6
                    break
                end
                println(i)
                println(cost_prev_it)
            end
        end
        println("finished: d = ", depth)
        println(costs[end])
        push!(it_costs,costs[end])
        println(coefficients)
    end
    println(it_costs)
    println(minimum(it_costs))
    push!(costs_final, minimum(it_costs))
end
costs_final

InterruptException: InterruptException:

In [150]:
coeffs = [0.8328194938452032, -0.8105522750655366, -0.03947596969773574, 0.8150417127415052, 0.9355586497841004, -0.819672727969885, 0.15908257377596594, 0.8969730569787527, 1.0867987077197083, -0.12382796442095499, -0.6769159418813943, 1.0246426230169303, -0.5084405635279222, -1.0268972961310772, 0.42484396266977487, -0.30984213118061454, -0.44195548463552226, -0.6327731382009563, 0.2470520241498077, -0.9046312383937988, 0.5670153531055953, 1.3215582797100167, 0.8729333306332443, -0.78886152481934, 0.3445569548035575, -1.2206696780026396, -0.25471100801485647, -0.3585072716256307, 0.6870581315744397, 0.6545040107617951, -1.0221685943730465, 0.3343131234446875, -0.08486885305074147, 1.16534326489163, -0.08178072747870009, -0.17367128231750495, 0.7301925883287174, 0.7960381150713036, -0.2445176384048139, 0.7450024213033812, -1.2896520166163863, -0.2511261526654848, 0.18899633152030276, -0.3134353271761458, 0.040134442582384806, 0.7718476862874823, -0.4457105371804348, -1.2658170397923836, -0.578778138744985, -0.27405676265384593, 0.6995176637177795, 0.35946077517978764, 1.0020099923271824, 0.9306624492297949, 0.9741238940562686, 1.1608927613144042, -0.5310290099006071, 0.8115833948525197, -0.7784531065547783, 0.7352221959115539, 0.5417078187690723, -0.21113457049160275, 0.5103730707595793, 0.7551026732286986, -1.7846390041521236, 0.8631165439769484, -0.5498701334703305, -0.26861474853829703, -0.27455529220647396, -0.07688130726216859, -0.5631568455670428, 0.8533900226411945, 2.8067345186309858, 0.6785936932829509, 3.3943749838043784, -2.805214370969844, -1.4757375492498623, 0.29692768551759546, 2.281415636662691, 2.160137328769026, -1.307321749251418, 1.033561682283, 2.3601394944015057, -2.696457234103049, 2.4969210954848147, -1.906381025598878, 2.2434123597180444, -1.9770399537034882, 1.690842335222719, 1.535746792078091, 1.8604714721583955, -1.3743895036802214, -1.291663669776218, -0.762668513556034, -1.3646714887473783, -0.7879082846775004, -0.9459496839509586, -1.5081193660443275, 2.163188081396188, -2.0477877934412, 1.8200709265036517, -2.7617245086967235, 1.4249304092894297, 0.24880088065636674, -1.2535266524595052, -2.0585026020721804, -4.576841695056939, 3.5387398533822982, -0.07531806514326735, 1.8453317273419196, 0.42767796713041867, 0.7150419102279323, 0.7474552662694737, -1.6860907528015776, -0.2962280834604321, -2.386788332449464, -1.1771757193252919, 0.09208258390797398, -0.8624876874455844, 0.5131833036461091, 0.6136063613800034, -2.0172431784872105, 1.9575955203123583, 1.3468300140519078, -0.7787319428945259, -2.5374797856832374, -1.9401420236852418, 2.0676338346841168, 0.7064923766586017, 1.1338034698609585, 2.9969098154086633, 1.0459448874008392, 3.4682585115304194, -3.9796395422002675, -0.8624504151631956, 1.4625786737059918, 0.8235301000679373, 2.5250726036878977, 2.085654990962269, -0.17147063484675754, 0.7961241933337927, 2.6289885365160464, 0.6582125941103614, -2.531971259434754]
f(x) = cost(x, X_gate, N*2)
println(f(coeffs))

opt_perturbed = ComplexF64[]

for coeff in coeffs
    perturbed = coeff + (rand()*0.002-0.001)
    push!(opt_perturbed,perturbed)
end

f(opt_perturbed)

0.8889226442425289


0.8889281184515271